In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import *
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
#extract development and evaulation
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")

In [ ]:
condition = (df['Current language used for work/school'] == np.unique(df_eval['Current language used for work/school'].values)[0]) & (df['First Language spoken'] == np.unique(df_eval['First Language spoken'].values)[0]) & (df['Self-reported fluency level '] == np.unique(df_eval['Self-reported fluency level '].values)[0])
df = df.loc[condition]

In [ ]:
df['intent'] = df['action']+ df['object']
y = df['intent']



In [ ]:
label_encoder(df, 'gender')
label_encoder(df_eval, 'gender')

In [ ]:
#dropping irrelevant columns
# df.drop(columns=["Self-reported fluency level ","First Language spoken", "Current language used for work/school","action","object", "intent"], inplace=True)
df.drop(columns=["action","object", "intent"], inplace=True)

In [ ]:
# #dropping irrelevant columns
# df_eval.drop(columns=["Self-reported fluency level ","First Language spoken", "Current language used for work/school"], inplace=True)

In [ ]:
label_encoder(df, 'gender')
label_encoder(df_eval, 'gender')

In [ ]:
label_encoder(df, 'ageRange')
label_encoder(df_eval, 'ageRange')

In [ ]:
audio_feature_extraction(df)
audio_feature_extraction(df_eval)

In [ ]:
import librosa
import numpy as np

def remove_silence(y, sr, threshold=0.01):
    rmse = librosa.feature.rms(y=y, frame_length=2048, hop_length=512)
    y_silent = np.array([0 if val <= threshold else val for val in rmse[0]])
    y_without_silence = y[y_silent != 0]
    sr_without_silence = sr
    return y_without_silence, sr_without_silence

y, sr = df["data"][0], df["rate"][0]
y_without_silence, sr_without_silence = remove_silence(y, sr)


In [ ]:
import librosa
import numpy as np

def extract_spectrogram(dataframe):
    audio_paths = dataframe["path"].values
    spectrograms = []
    for data in dataframe['data']:
        
        # Compute the spectrogram
        spectrogram = np.abs(librosa.stft(data))
        
        # Split the spectrogram into 20 chunks
        n_chunks = 20
        chunks = np.array_split(spectrogram, n_chunks)
        
        # Append the chunks to the list of spectrograms
        spectrograms.append(chunks)
    
    # Add the spectrogram chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_spectrogram_" + str(i)] = [np.mean(chunk[i]) for chunk in spectrograms]
        dataframe["std_spectrogram_" + str(i)] = [np.std(chunk[i]) for chunk in spectrograms]
    return dataframe


In [ ]:
extract_spectrogram(df)

In [ ]:
extract_spectrogram(df_eval)

In [ ]:
def extract_mfcc(dataframe):
    mfcc_array = []
    for data, rate in zip(dataframe['data'], dataframe['rate']):
        # Load the audio file
        mfcc = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=30)        
        # Split the spectrogram into 20 chunks
        n_chunks = 20
        chunks = np.array_split(mfcc, n_chunks)
        
        # Append the chunks to the list of spectrograms
        mfcc_array.append(chunks)
    
    # Add the spectrogram chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_mfcc_" + str(i)] = [np.mean(chunk[i]) for chunk in mfcc_array]
        dataframe["std_mfcc_" + str(i)] = [np.std(chunk[i]) for chunk in mfcc_array]
    return dataframe



In [ ]:
extract_mfcc(df)

In [ ]:
extract_mfcc(df_eval)

In [ ]:
rmse_feature(df)
rmse_feature(df_eval)

time_domain_1D(df,'rmse')
time_domain_1D(df_eval,'rmse')

In [ ]:
sro_feature(df)
sro_feature(df_eval)
time_domain_1D(df,'spectral_rolloff')
time_domain_1D(df_eval,'spectral_rolloff')

In [ ]:
spectral_flatness(df)
spectral_flatness(df_eval)
time_domain_1D(df,'sf')
time_domain_1D(df_eval,'sf')

In [ ]:
zcr_feature(df)
zcr_feature(df_eval)

In [ ]:
def extract_chroma(dataframe):
    chroma_array = []
    for data, rate in zip(dataframe['data'], dataframe['rate']):
        # Load the audio file
        chroma = librosa.feature.chroma_stft(y=data, sr=rate, n_chroma=7)       
        # Split the spectrogram into 20 chunks
        n_chunks = 20
        chunks = np.array_split(chroma, n_chunks)
        
        # Append the chunks to the list of spectrograms
        chroma_array.append(chunks)
    
    # Add the spectrogram chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_chroma_" + str(i)] = [np.mean(chunk[i]) for chunk in chroma_array]
        dataframe["std_chroma_" + str(i)] = [np.std(chunk[i]) for chunk in chroma_array]
    return dataframe



In [ ]:
extract_chroma(df)

In [ ]:
extract_chroma(df_eval)

In [ ]:
def extract_tonnetz(dataframe):
    tonnetz_array = []
    for data, rate in zip(dataframe['data'], dataframe['rate']):
        # Load the audio file
        tonnetz = librosa.feature.tonnetz(y=data, sr=rate)
        # Split the spectrogram into 20 chunks
        n_chunks = 20
        chunks = np.array_split(tonnetz, n_chunks)
        
        # Append the chunks to the list of spectrograms
        tonnetz_array.append(chunks)
    
    # Add the spectrogram chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_tonnetz_" + str(i)] = [np.mean(chunk[i]) for chunk in tonnetz_array]
        dataframe["std_tonnetz_" + str(i)] = [np.std(chunk[i]) for chunk in tonnetz_array]
    return dataframe



In [ ]:
extract_tonnetz(df)
extract_tonnetz(df_eval)

In [ ]:
def extract_spectral_contrast(dataframe):
    spectral_contrast_array = []
    for data, rate in zip(dataframe['data'], dataframe['rate']):
        # Load the audio file
        spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=rate)
        # Split the spectrogram into 20 chunks
        n_chunks = 20
        chunks = np.array_split(spectral_contrast, n_chunks)
        
        # Append the chunks to the list of spectrograms
        spectral_contrast_array.append(chunks)
    
    # Add the spectrogram chunks to the dataframe as new columns
    for i in range(n_chunks):
        dataframe["mean_tspectral_contrast_" + str(i)] = [np.mean(chunk[i]) for chunk in spectral_contrast_array]
        dataframe["std_spectral_contrast_" + str(i)] = [np.std(chunk[i]) for chunk in spectral_contrast_array]
    return dataframe



In [ ]:
extract_spectral_contrast(df)
extract_spectral_contrast(df_eval)

In [ ]:
df_eval.drop(columns=['Id','ageRange','data','speakerId','path', 'rate'],inplace= True)
df.drop(columns=['Id','ageRange','data','speakerId','path', 'rate'],inplace= True)

In [ ]:
df

In [ ]:
X = df.copy()
X_eval = df_eval.copy()

In [ ]:
X

In [ ]:
X.drop(columns=['rmse','sf','spectral_rolloff'], inplace=True)
X_eval.drop(columns=['rmse','sf','spectral_rolloff'], inplace=True)

In [ ]:
X = X.dropna(axis=1, how='all')
X_eval = X_eval.dropna(axis=1, how='all')


In [ ]:


X.to_csv('x.csv',index=False)

        
X_eval.to_csv('x_eval.csv', index=False)

In [ ]:
X_eval = pd.read_csv('x_eval.csv')
X = pd.read_csv('x.csv')

In [ ]:

# def calculate_class_counts(X, y):
#     class_counts = {}
#     for target in set(y):
#         class_counts[target] = y.tolist().count(target)
#     return class_counts

# print(calculate_class_counts(X_new, y_new))

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Create a copy of the dataframe
X_scaled = X.copy()
X_eval_scaled = X_eval.copy()

# Extract the column names
column_names = X.columns

# Create the StandardScaler object
scaler = StandardScaler()

# Fit and transform the data
X_scaled[column_names] = scaler.fit_transform(X[column_names])
# X_eval_scaled[column_names] = scaler.transform(X_eval[column_names])


In [ ]:
from sklearn.decomposition import PCA

def select_top_n_features(X_train, X_test, threshold):
    pca = PCA()
    
    # Fit PCA on the training data
    pca.fit(X_train)
    
    # Get the variance ratio for each principal component
    variance_ratio = pca.explained_variance_ratio_
    n_components = len(variance_ratio)
    
    # Find the number of components that explain at least the threshold of variance
    explained_variance = 0
    for i, ratio in enumerate(variance_ratio):
        explained_variance += ratio
        if explained_variance >= threshold:
            n_components = i + 1
            break
    print(f"Number of components that explain at least {threshold} of variance : {n_components}")
    # Initialize the PCA object with the number of components
    pca = PCA(n_components=n_components)
    
    # Fit PCA on the training data
    pca.fit(X_train)
    # Transform the training and test data to the first n principal components
    X_train_top_n_features = pca.transform(X_train)
    X_test_top_n_features = pca.transform(X_test)
    #Return the top features and the variance ratio
    return X_train_top_n_features, X_test_top_n_features


In [ ]:
X, X_eval =select_top_n_features(X_scaled, X_eval_scaled, 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split


def svm_model2(X_train, y_train, X_test):
    clf = SVC(C = 100, gamma = 0.01, kernel = 'rbf')
    # train the model on the training data
    clf.fit(X_train, y_train)
    # predict the target values for the test data
    # returning the y_predict
    return clf.predict(X_test)
y_pred = svm_model2(X, y, X_test)

In [ ]:
accuracy_calculator(y_test, y_pred)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def label_encodes(df, column_name):
    le = LabelEncoder()
    label = le.fit_transform(df[column_name])
    df.drop(column_name, axis=1, inplace=True)
    df[column_name] = label

label_encodes(X, 'Self-reported fluency level ')
label_encodes(X, 'First Language spoken')
label_encodes(X, 'Current language used for work/school')

In [ ]:
from sklearn.svm import SVC


def svm_model2(X_train, y_train, X_test):
    clf = SVC(kernel="rbf",C=10)
    # train the model on the training data
    clf.fit(X_train, y_train)
    # predict the target values for the test data
    # returning the y_predict
    return clf.predict(X_test)
y_pred = svm_model2(X_train, y_train, X_test)
svm_accuracy = accuracy_calculator(y_test, y_pred)

In [ ]:
svm_accuracy

In [ ]:
y_decoded = label_encoder.inverse_transform(y_pred)
pd.Series(y_decoded).to_csv('predictions.csv', index='Id')

In [ ]:
import numpy as np

# Create a list of all possible values for each attribute
n_estimators = np.arange(10, 100, 10)
max_depth = [None, 5, 10, 15, 20]
min_samples_split = np.arange(2, 20, 2)
min_samples_leaf = np.arange(1, 20, 2)
max_features = ['auto', 'sqrt', 'log2']

# Create a list of all possible combinations of the attributes
combinations = [(a,b,c,d,e) for a in n_estimators for b in max_depth for c in min_samples_split for d in min_samples_leaf for e in max_features]

# Select the first four combinations
selected_combinations = combinations[:4]

# Print the selected combinations
print(selected_combinations)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
accuracy = []

# Generate some sample data

for comb in combinations:

    # Initialize the Random Forest classifier
    rf = RandomForestClassifier(n_estimators = comb[0],max_depth=comb[1], min_samples_split = comb[2], min_samples_leaf=  comb[3], max_features= comb[4])

    # Fit the classifier to the data
    rf.fit(X_train, y_train)

    # Make predictions on new data
    y_pred = rf.predict(X_test)

    # Print the accuracy of the model
    print(comb)
    print("Accuracy:", rf.score(X_test, y_test))
    accuracy.append((rf.score(X_test, y_test),comb))



In [ ]:
y_decoded = label_encoder.inverse_transform(y_pred)
pd.Series(y_decoded).to_csv('predictions.csv', index='Id')

In [ ]:
np.max(accuracy)